# MomentOpt and parallel computing
## The inference problem
This example is a simplified illustration of the simulated method of moments, which is often used in Economics. The setting is the following. We have access to historical series, from which we can calculate empirical moments. Theory gives us a model, which rationalizes the observed historical series. The model we assume that true depends on some parameters, which we would like to estimate. While [maximizing the likelihood](https://en.wikipedia.org/wiki/Maximum_likelihood_estimation) could be an option, here we posit that the likelihood function is untractable. However, for any parameter values, we are able to generate a sample from the model. In this setting, one may use a [simulated method of moments](https://en.wikipedia.org/wiki/Method_of_simulated_moments) approach. This etimation strategy is based on the idea that, at the true parameter values, the observed moments (the one calculate from data) should be equal to the simulated moments (coming from our simulated sample).

## A simplified setting
Here, we are generating a sample from a 4-dimensional normal variable with unit variance. We rightly assume that the data comes from a multivariate normal distribution (our "model"). Our goal is to estimate the mean of the 4-dimensional distribution generating the sample. 
 
## Adding workers
If you start julia with several processors. The first step is to add "workers" to julia. A rule of thumb is to use as many workers as you have processors on your system.

In [1]:
# Current number of workers
#--------------------------
currentWorkers = nprocs()
println("Initial number of workers = $(currentWorkers)")
# I want to have 4 workers running
#--------------------------------
maxNumberWorkers = 4
while nprocs() < maxNumberWorkers
    addprocs(1)
end
# check the number of workers:
#----------------------------
currentWorkers = nprocs()
println("Number of workers = $(currentWorkers)")

Initial number of workers = 1
Number of workers = 4


## Historical series

In a real-world setting, historical series would be given to you. Here we generate them. Note that we use the macro @everywhere because we have several workers. 

In [4]:
@everywhere using MomentOpt
@everywhere using GLM
@everywhere using DataStructures
@everywhere using DataFrames
@everywhere using Plots
@everywhere using Distributions; plotlyjs()
#@everywhere pyplot()

srand(1234);

# Define true values of parameters
#---------------------------------
trueValues = OrderedDict("mu1" => [-1.0] , "mu2" => [1.0], "mu3" => [5.0], "mu4" => [-4.0])

mu = [trueValues["mu1"]; trueValues["mu2"]; trueValues["mu3"]; trueValues["mu4"]]

# Generate "real" data:
#-----------------------
# Guassian distribution with mean mu and unit variance
d = MvNormal(mu, ones(size(mu)))

# How many draws for our "real" data?
#------------------------------------
ndraws = 100000
empiricalSample = rand(d, ndraws)

4×100000 Array{Float64,2}:
 -0.132653   -0.135599  -0.497666  …  -2.17959   -1.84134  -2.06573
  0.0982562   3.21188    0.483016      0.463844   2.07502   2.40871
  4.50552     5.53281    4.4395        5.41874    4.80355   5.35252
 -4.90291    -4.27174   -4.01929      -3.5498    -4.59967  -3.41002

In [5]:
# Plot historical series:
#------------------------
# Every observation is 4-dimensional point.
# Plot component by component.
p = plot(empiricalSample[1,1:1000], label = "x1")
plot!(empiricalSample[2,1:1000], label = "x2")
plot!(empiricalSample[3,1:1000], label = "x3")
plot!(empiricalSample[4,1:1000], label = "x4")
display(p)

# Calculate empirical moments 
#----------------------------
empiricalMoments =  mean(empiricalSample,2)
println("Empirical Moments = $(empiricalMoments)")

Empirical Moments = [-1.00096; 1.0002; 5.00753; -4.00131]


## Set up a MomentOpt problem

In [6]:
#---------------------------------------------------------------------------------------------------------
# Julien Pascal
# https://julienpascal.github.io/
# last edit: 21/08/2018
#
# Julia script that shows how the simulated method of moments can be used in
# a simple setting: estimation of the mean of a Normal r.v.
# This version was built to be executed with several processors
#----------------------------------------------------------------------------------------------------------

#------------------------------------------------
# Options
#-------------------------------------------------
# Boolean: do you want to save the plots to disk?
savePlots = true

#------------------------
# initialize the problem:
#------------------------
# Specify the initial values for the parameters, and their support:
#------------------------------------------------------------------
pb = OrderedDict("p1" => [0.2,-3,3] , "p2" => [-0.2,-2,2], "p3" => [0.1,0,10], "p4" => [-0.1,-10,0])
# Specify moments to be matched + subjective weights:
#----------------------------------------------------
# use moments calculated above:
# give same weight to each parameter
moms = DataFrame(name=["mu1","mu2","mu3", "mu4"],value=[empiricalMoments[1], empiricalMoments[2], empiricalMoments[3], empiricalMoments[4]], weight=ones(4))


# objfunc_normal(ev::Eval)
#
# GMM objective function to be minized.
# It returns a weigthed distance between empirical and simulated moments
#
@everywhere function objfunc_normal(ev::Eval; verbose = false)

    start(ev)


    # when running in parallel, display worker's id:
    #-----------------------------------------------
    if verbose == true
        if nprocs() > 1
          println(myid())
        end
    end

    # extract parameters from ev:
    #----------------------------
    mu  = collect(values(ev.params))

    # compute simulated moments
    #--------------------------
    # Monte-Carlo:
    #-------------
    ns = 100000 #number of i.i.d draws from N([mu], sigma)
    #initialize a multivariate normal N([mu], sigma)
    #mu is a four dimensional object
    #sigma is set to be the identity matrix
    sigma = [1.0 ;1.0; 1.0; 1.0]
    # draw ns observations from N([mu], sigma):
    randMultiNormal = MomentOpt.MvNormal(mu,MomentOpt.PDiagMat(sigma))
    # calculate the mean of the simulated data
    simM            = mean(rand(randMultiNormal,ns),2)
    # store simulated moments in a dictionary
    simMoments = Dict(:mu1 => simM[1], :mu2 => simM[2], :mu3 => simM[3], :mu4 => simM[4])

    # Calculate the weighted distance between empirical moments
    # and simulated ones:
    #-----------------------------------------------------------
    v = Dict{Symbol,Float64}()
    for (k, mom) in dataMomentd(ev)
        # If weight for moment k exists:
        #-------------------------------
        if haskey(MomentOpt.dataMomentWd(ev), k)
            # divide by weight associated to moment k:
            #----------------------------------------
            v[k] = ((simMoments[k] .- mom) ./ MomentOpt.dataMomentW(ev,k)) .^2
        else
            v[k] = ((simMoments[k] .- mom) ) .^2
        end
    end

    # Set value of the objective function:
    #------------------------------------
    setValue(ev, mean(collect(values(v))))

    # also return the moments
    #-----------------------
    setMoment(ev, simMoments)

    # flag for success:
    #-------------------
    ev.status = 1

    # finish and return
    finish(ev)

    return ev
end



# Initialize an empty MProb() object:
#------------------------------------
mprob = MProb()

# Add structural parameters to MProb():
# specify starting values and support
#--------------------------------------
addSampledParam!(mprob,pb)

# Add moments to be matched to MProb():
#--------------------------------------
addMoment!(mprob,moms)

# Attach an objective function to MProb():
#----------------------------------------
addEvalFunc!(mprob, objfunc_normal)


# estimation options:
#--------------------
# number of iterations for each chain
niter = 1000
# number of chains
# nchains = nprocs()
nchains = 4

opts = Dict("N"=>nchains,
        "maxiter"=>niter,
        "maxtemp"=> 5,
        "coverage"=>0.025,
        "sigma_update_steps"=>10,
        "sigma_adjust_by"=>0.01,
        "smpl_iters"=>1000,
        "parallel"=>true,
        "maxdists"=>[0.05 for i in 1:nchains],
        "mixprob"=>0.3,
        "acc_tuner"=>12.0,
        "animate"=>false)



#---------------------------------------
# Let's set-up and run the optimization
#---------------------------------------
# set-up BGP algorithm:
MA = MAlgoBGP(mprob,opts)

# run the estimation:
@time MomentOpt.runMOpt!(MA)

# show a summary of the optimization:
@show MomentOpt.summary(MA)


12:27:47:INFO:Main:Starting estimation loop.
12:35:51:WARN:Main:could not find 'filename' and did not save
12:35:51:INFO:Main:Done with estimation after 8.1 minutes


486.596021 seconds (272.97 M allocations: 14.511 GiB, 1.94% gc time)


,id,acc_rate,perc_exchanged,exchanged_most_with,best_val
1,1,0.507722,48.3,4,0.00856991
2,2,0.327434,43.6,1,0.00856991
3,3,0.246599,41.2,1,0.00856991
4,4,0.209262,41.8,1,0.017862


MomentOpt.summary(MA) = 4×5 DataFrames.DataFrame
│ Row │ id │ acc_rate │ perc_exchanged │ exchanged_most_with │ best_val   │
├─────┼────┼──────────┼────────────────┼─────────────────────┼────────────┤
│ 1   │ 1  │ 0.507722 │ 48.3           │ 4                   │ 0.00856991 │
│ 2   │ 2  │ 0.327434 │ 43.6           │ 1                   │ 0.00856991 │
│ 3   │ 3  │ 0.246599 │ 41.2           │ 1                   │ 0.00856991 │
│ 4   │ 4  │ 0.209262 │ 41.8           │ 1                   │ 0.017862   │


## Inference
When using the BGP algorithm, inference can be done using the first chain. Other chains are used to explore the state space and help to exit local minima, but they are not meant to be used for inference. I discard the first 10th observations to get rid of the influence of the starting values (visual inspection of the first chain suggests that the stationary part of the Markov chain was reached at this stage). I then report the quasi posterior mean and median for each parameter. As reported below, we are quite close to the true values.  

In [7]:
# Plot histograms for the first chain, the one with which inference should be done.
# Other chains are used to explore the space and avoid local minima
#-------------------------------------------------------------------------------
p1 = histogram(MA.chains[1])
display(p1)

if savePlots == true
    savefig(p1, joinpath(pwd(),"histogram_chain1.svg"))
end

# Plot the realization of the first chain
#----------------------------------------
p2 = plot(MA.chains[1])
if savePlots == true
    savefig(p2, joinpath(pwd(),"history_chain_1.svg"))
end
display(p2)

`AssetRegistry.jl` instead to register assets directory


In [17]:

# Realization of the first chain:
#-------------------------------
dat_chain1 = MomentOpt.history(MA.chains[1])

# discard the first 10th of the observations ("burn-in" phase):
#--------------------------------------------------------------
dat_chain1[round(Int, niter/10):niter, :]

# keep only accepted draws:
#--------------------------
dat_chain1 = dat_chain1[dat_chain1[:accepted ].== true, : ]

# create a list with the parameters to be estimated
parameters = [Symbol(String("mu$(i)")) for i=1:4]
# list with the corresponding priors:
#------------------------------------
estimatedParameters = [Symbol(String("p$(i)")) for i=1:4]


# Quasi Posterior mean and quasi posterior median for each parameter:
#-------------------------------------------------------------------
for (estimatedParameter, param) in zip(estimatedParameters, parameters)

  println("Quasi posterior mean for $(String(estimatedParameter)) = $(mean(dat_chain1[estimatedParameter]))")
  println("Quasi posterior median for $(String(estimatedParameter)) = $(median(dat_chain1[estimatedParameter]))")
  println("True value = $(trueValues[String(param)][])")
  println("abs(True value - Quasi posterior median)  = $(abs(median(dat_chain1[estimatedParameter]) - trueValues[String(param)][])) \n")

end

Quasi posterior mean for p1 = -0.9933866913736091
Quasi posterior median for p1 = -1.0042108996171293
True value = -1.0
abs(True value - Quasi posterior median)  = 0.004210899617129282 

Quasi posterior mean for p2 = 0.8741783094683604
Quasi posterior median for p2 = 0.8783767138071625
True value = 1.0
abs(True value - Quasi posterior median)  = 0.12162328619283747 

Quasi posterior mean for p3 = 4.884588560515209
Quasi posterior median for p3 = 4.935585186131482
True value = 5.0
abs(True value - Quasi posterior median)  = 0.06441481386851766 

Quasi posterior mean for p4 = -3.891640334309565
Quasi posterior median for p4 = -3.955359653906345
True value = -4.0
abs(True value - Quasi posterior median)  = 0.04464034609365486 



## Safety checks
We can also make sure that the objective function is "well-behaved" in a neighborhood of the solution (i.e. with no "flat region") using slices. The graph below shows that there is no flat region in the neighborhood of the solution, suggesting (at least) local identification of the parameters. In this toy example, we know we have global identification, but in "real-life" application global identification may be hard to prove analytically. 

In [18]:
# plot slices of objective function
# grid with 20 points
#-----------------------------------
s = doSlices(mprob,20)

# plot slices of the objective function:
#---------------------------------------
p = MomentOpt.plot(s,:value)
display(p)

if savePlots == true
    Plots.savefig(p, joinpath(pwd(),"slices_Normal.svg"))
end



# Produce more precise plots with respect to each parameter:
#-----------------------------------------------------------
for symbol in parameters

  p = MomentOpt.plot(s,symbol)
  display(p)
    
  if savePlots == true
      Plots.savefig(p, joinpath(pwd(),"slices_Normal_$(String(symbol)).svg"))
  end
    

end

12:40:37:INFO:Main:slicing along p1
12:40:41:INFO:Main:slicing along p2
12:40:45:INFO:Main:slicing along p3
12:40:49:INFO:Main:slicing along p4
12:40:53:INFO:Main:done after 0.0 minutes
